In [1]:
from vllm import LLM

llm = LLM(
    model='Qwen/Qwen3-VL-Embedding-2B',
    runner='pooling',
    max_model_len=4096,
    gpu_memory_utilization=0.85,
    quantization="mxfp4"
)

INFO 01-27 21:59:11 [utils.py:263] non-default args: {'runner': 'pooling', 'max_model_len': 4096, 'gpu_memory_utilization': 0.85, 'disable_log_stats': True, 'quantization': 'mxfp4', 'model': 'Qwen/Qwen3-VL-Embedding-2B'}
INFO 01-27 21:59:12 [model.py:859] Resolved `--convert auto` to `--convert embed`. Pass the value explicitly to silence this message.
INFO 01-27 21:59:12 [model.py:530] Resolved architecture: Qwen3VLForConditionalGeneration
INFO 01-27 21:59:12 [model.py:1545] Using max model len 4096
INFO 01-27 21:59:12 [scheduler.py:229] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 01-27 21:59:15 [vllm.py:630] Asynchronous scheduling is enabled.
INFO 01-27 21:59:15 [vllm.py:637] Disabling NCCL for DP synchronization when using async scheduling.
WARNING 01-27 21:59:15 [vllm.py:744] Pooling models do not support full cudagraphs. Overriding cudagraph_mode to PIECEWISE.
(EngineCore_DP0 pid=121944) INFO 01-27 21:59:18 [core.py:97] Initializing a V1 LLM engine (v0.14.0)

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


(EngineCore_DP0 pid=121944) INFO 01-27 21:59:32 [default_loader.py:291] Loading weights took 1.07 seconds
(EngineCore_DP0 pid=121944) INFO 01-27 21:59:33 [gpu_model_runner.py:3905] Model loading took 4.31 GiB memory and 2.921968 seconds
(EngineCore_DP0 pid=121944) INFO 01-27 21:59:34 [gpu_model_runner.py:4715] Encoder cache will be initialized with a budget of 8192 tokens, and profiled with 2 video items of the maximum feature size.
(EngineCore_DP0 pid=121944) INFO 01-27 21:59:48 [backends.py:644] Using cache directory: /home/administrator/.cache/vllm/torch_compile_cache/b239d35871/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=121944) INFO 01-27 21:59:48 [backends.py:704] Dynamo bytecode transform time: 7.48 s
(EngineCore_DP0 pid=121944) INFO 01-27 21:59:53 [backends.py:226] Directly load the compiled graph(s) for compile range (1, 8192) from the cache, took 1.156 s
(EngineCore_DP0 pid=121944) INFO 01-27 21:59:53 [monitor.py:34] torch.compile takes 8.64 s in total
(Eng

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:01<00:00, 26.16it/s]


(EngineCore_DP0 pid=121944) INFO 01-27 21:59:57 [gpu_model_runner.py:4856] Graph capturing finished in 3 secs, took 0.31 GiB
(EngineCore_DP0 pid=121944) INFO 01-27 21:59:57 [core.py:273] init engine (profile, create kv cache, warmup model) took 24.07 seconds
INFO 01-27 21:59:58 [llm.py:347] Supported tasks: ['embed', 'token_embed']


In [2]:
from sqlalchemy import create_engine, text
from e_commerce_recommendation.configs.settings import settings
import polars as pl

DATABASE_URL = settings.DATABASE_URL

engine = create_engine(DATABASE_URL)
categories = pl.read_database(
    query="SELECT * FROM categories",
    connection=engine
).sort("name")
categories

id,name
i64,str
104,"""3d printers"""
407,"""Adult Diapers and Incontinence"""
360,"""Amazon artisan"""
402,"""Ayurveda Products"""
251,"""Baby Jewelery and Jewelery"""
…,…
236,"""women's handbags"""
502,"""women's health family planning"""
410,"""women's lehenga choli"""


In [8]:
categories[265]

id,name
i64,str
265,"""software"""


In [5]:
categories.write_database(
    table_name="categories",
    connection=settings.SAMPLE_DATABASE_URL,
    if_table_exists="append",
)

508

In [6]:
categories['name'].to_list()

['groceries',
 'smart home security & lighting',
 'string instruments',
 'computer screws',
 'home use medical supplies & equipment',
 'fragrances',
 'smart home',
 'automotive tools equipment',
 "Men's coats, jackets and vests",
 'electrical',
 'beading & jewelry making',
 'bath body',
 'computer memory',
 'consoles',
 "men's fashion",
 'computer audio video accessories',
 'action sports',
 'home decor',
 'cable accessories',
 'camera & photo accessories',
 'indoor lighting',
 'accessories & supplies',
 "men's jewelry",
 'backpacks',
 'collectible toys',
 'garden decor',
 'heating cooling air quality',
 'car & motorbike',
 'ironing & steamers',
 'bathroom furniture',
 'household cleaning',
 'accessories',
 "kids' party supplies",
 'home kitchen',
 'men',
 'garden tools & watering equipment',
 "men's wallets",
 'beauty',
 'boys',
 "kids' play cars & race cars",
 'bathroom linen',
 'computer monitor accessories',
 'computers & tablets',
 'travel duffel bags',
 'exercise fitness equipmen

In [29]:
default_instruction = "Represent the user's input for multi-modal ecommerce products recommendation. focus more on product's detailing"
prompts = []
for text in categories['name'].to_list():
    prompts.append(f"<|im_start|>system\n{default_instruction}<|im_end|>\n<|im_start|>user\n{text}<|im_end|>\n<|im_start|>assistant\n")
prompts

["<|im_start|>system\nRepresent the user's input for multi-modal ecommerce products recommendation. focus more on product's detailing<|im_end|>\n<|im_start|>user\n3d printers<|im_end|>\n<|im_start|>assistant\n",
 "<|im_start|>system\nRepresent the user's input for multi-modal ecommerce products recommendation. focus more on product's detailing<|im_end|>\n<|im_start|>user\nAdult Diapers and Incontinence<|im_end|>\n<|im_start|>assistant\n",
 "<|im_start|>system\nRepresent the user's input for multi-modal ecommerce products recommendation. focus more on product's detailing<|im_end|>\n<|im_start|>user\nAmazon artisan<|im_end|>\n<|im_start|>assistant\n",
 "<|im_start|>system\nRepresent the user's input for multi-modal ecommerce products recommendation. focus more on product's detailing<|im_end|>\n<|im_start|>user\nAyurveda Products<|im_end|>\n<|im_start|>assistant\n",
 "<|im_start|>system\nRepresent the user's input for multi-modal ecommerce products recommendation. focus more on product's 

In [30]:
embedding_outputs = llm.embed(prompts=prompts)

Adding requests:   0%|          | 0/508 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/508 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [33]:
req_cat = "computer"
output = llm.embed(f"<|im_start|>system\n{default_instruction}<|im_end|>\n<|im_start|>user\n{req_cat}<|im_end|>\n<|im_start|>assistant\n")

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [35]:
vectors_as_lists = [str(output.outputs.embedding) for output in embedding_outputs]

category_embeddings_df = pl.DataFrame(
    {"id": categories["id"], "embedding": vectors_as_lists}
)

category_embeddings_df

id,embedding
i64,str
104,"""[0.011400449089705944, -0.0432…"
407,"""[-0.00875409971922636, -0.0018…"
360,"""[0.007562330458313227, -0.0488…"
402,"""[0.005640950053930283, -0.0300…"
251,"""[0.025100374594330788, -0.0582…"
…,…
236,"""[-0.03475501015782356, -0.0420…"
502,"""[-0.018534356728196144, -0.025…"
410,"""[0.009683012962341309, -0.0716…"


In [37]:
str(output[0].outputs.embedding)

'[-0.00422302819788456, -0.015760943293571472, -0.04404015094041824, 0.0030353013426065445, 0.0018758539808914065, -0.01757081411778927, -0.06606022268533707, 0.053692784160375595, -0.024282410740852356, -0.027148038148880005, 0.020361028611660004, -0.02925955131649971, 0.015459299087524414, 0.04072205722332001, 0.04645330831408501, -0.020210206508636475, 0.02382994443178177, 0.01930527202785015, 0.08385727554559708, -0.02910872921347618, -0.01523306593298912, 0.02910872921347618, 0.035292450338602066, -0.02232171967625618, -0.034085869789123535, 0.0028090677224099636, -0.0036385911516845226, -0.001461092266254127, 0.047961533069610596, -0.03770560771226883, 0.02503652311861515, -0.039515476673841476, 0.028354616835713387, -0.010255925357341766, 0.014931420795619488, -0.006673892494291067, 0.026846392080187798, -0.014403542503714561, 0.010783803649246693, 0.00656077591702342, 0.0056558409705758095, 0.020511850714683533, 0.006146014202386141, -0.003431210294365883, 0.043738506734371185,

In [34]:
vectors_as_lists = [output.outputs.embedding for output in embedding_outputs]

category_embeddings_df = pl.DataFrame(
    {"id": categories["id"], "embedding": vectors_as_lists, "similarity": np.array(vectors_as_lists) @ np.array(output[0].outputs.embedding).T }
)

category_embeddings_df.join(categories, on="id").sort("similarity", descending=True).head(10)

id,embedding,similarity,name
i64,list[f64],f64,str
101,"[-0.0043, -0.006469, … 0.005347]",0.881768,"""desktop computers"""
119,"[-0.00054, -0.020656, … 0.012348]",0.865188,"""laptops computers"""
42,"[-0.008611, -0.016327, … 0.017893]",0.841101,"""computers & tablets"""
120,"[-0.006405, -0.023981, … 0.004617]",0.8253,"""computer accessories"""
131,"[-0.006405, -0.023981, … 0.004617]",0.8253,"""computer accessories"""
91,"[-0.015468, -0.017073, … 0.000037]",0.803539,"""computer monitors"""
411,"[-0.033402, -0.028206, … -0.006272]",0.784965,"""computer cases"""
69,"[-0.003752, -0.013906, … 0.010301]",0.768417,"""computer printers"""
437,"[0.027237, 0.006846, … -0.013253]",0.762769,"""CPU"""


In [ ]:
# cosine similarity
import numpy as np
def cosine_similarity(x):
    np.array(category_embeddings_df['embedding'][0])

array([-0.00145667, -0.06108978, -0.01751626, ...,  0.00962671,
        0.03184775,  0.01686483], shape=(2048,))

In [36]:
category_embeddings_df.write_database(
    table_name="category_embeddings",
    connection=settings.SAMPLE_DATABASE_URL,
    if_table_exists="append"
)

508

In [27]:
pl.DataFrame([embedding.outputs.embedding for embedding in embedding_outputs])

column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,column_9,column_10,column_11,column_12,column_13,column_14,column_15,column_16,column_17,column_18,column_19,column_20,column_21,column_22,column_23,column_24,column_25,column_26,column_27,column_28,column_29,column_30,column_31,column_32,column_33,column_34,column_35,column_36,…,column_471,column_472,column_473,column_474,column_475,column_476,column_477,column_478,column_479,column_480,column_481,column_482,column_483,column_484,column_485,column_486,column_487,column_488,column_489,column_490,column_491,column_492,column_493,column_494,column_495,column_496,column_497,column_498,column_499,column_500,column_501,column_502,column_503,column_504,column_505,column_506,column_507
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.010744,0.006176,0.025796,-0.019291,-0.016831,-0.020187,0.004134,0.009687,0.013782,0.018354,0.005908,-0.035199,-0.019965,-0.003527,0.006777,0.005535,-0.006354,0.002216,-0.019259,-0.035729,0.015217,-0.022355,0.054739,-0.075635,0.01943,0.019084,0.029723,-0.022144,-0.008991,0.004199,0.050433,-0.006241,0.002256,0.006978,-0.018011,0.01037,0.020336,…,-0.01022,0.005719,0.008907,-0.032962,0.004951,-0.024585,-0.00536,0.011513,-0.018395,-0.011676,0.028117,-0.009043,-0.010949,-0.00146,-0.001167,-0.039795,-0.028689,0.012945,-0.015934,0.022075,-0.05694,-0.028346,-0.017959,0.007693,0.016914,-0.009267,0.006687,-0.037396,0.028949,-0.002178,0.023552,-0.015501,-0.052284,-0.084848,0.018335,0.024995,-0.008736
0.025526,-0.044761,-0.051297,-0.025194,0.015112,-0.065751,-0.025496,-0.012485,-0.032086,-0.022635,-0.102203,-0.045194,-0.024131,-0.047476,-0.059351,-0.054315,-0.049088,-0.05421,-0.049152,-0.080172,-0.039787,-0.056173,-0.086991,-0.044508,-0.053762,-0.047858,0.014494,-0.033507,-0.034521,-0.001269,0.020989,-0.0928,-0.020814,0.036421,-0.058625,-0.010149,-0.050768,…,-0.026239,-0.019692,-0.008691,-0.066214,-0.039315,-0.056872,-0.048097,-0.04576,-0.037828,-0.045543,-0.026369,-0.032699,-0.037705,-0.04038,-0.007324,-0.026078,-0.006262,-0.089097,-0.057707,0.021931,-0.027743,-0.030527,-0.064537,-0.107039,0.005339,-0.050203,-0.057835,-0.044582,-0.06667,-0.019139,-0.064153,-0.027288,-0.045566,-0.027399,-0.082652,-0.006174,-0.056437
-0.054225,0.040111,0.028596,-0.025914,0.009167,0.011752,0.040793,-0.011265,0.053671,0.006928,0.022892,0.022307,-0.028152,-0.016348,0.02297,-0.022287,0.034275,0.01553,-0.022277,-0.011038,-0.002617,0.011321,0.007286,-0.010691,-0.031118,-0.000048,0.001582,0.026369,0.066443,0.028627,0.020406,0.012701,-0.011644,-0.0153,0.001622,-0.016106,0.012043,…,0.016961,0.038805,0.056887,0.021397,0.027934,0.028732,-0.042592,0.024493,0.001483,0.005403,0.015297,0.021992,0.017257,0.017378,0.023795,0.015602,-0.034951,0.026758,-0.000185,-0.010677,0.049096,-0.006178,0.037233,-0.010407,-0.021644,-0.002271,0.01764,-0.022878,0.01272,-0.051989,0.009825,0.021716,0.011684,0.012963,0.001155,0.020206,0.010162
-0.031006,-0.004487,-0.032282,-0.019003,-0.008272,0.047872,-0.002167,0.011696,-0.012251,-0.021619,-0.030129,-0.005287,-0.023556,-0.002855,-0.003032,0.010701,-0.014015,-0.006007,-0.015737,0.006027,-0.014994,-0.023071,-0.034027,0.006327,0.01797,-0.044041,-0.01957,-0.007648,0.009461,0.014825,0.000148,-0.022172,-0.007277,-0.024181,-0.028146,-0.018166,0.026394,…,0.022035,0.016145,-0.008691,0.019806,-0.010568,-0.036138,0.010213,-0.044,-0.005971,-0.009029,-0.00834,0.000288,-0.019143,0.011465,-0.066924,-0.006746,0.023447,-0.03182,-0.01335,-0.03102,0.017358,-0.019624,-0.015988,0.002063,-0.036982,-0.01547,-0.033833,-0.056314,-0.026902,0.010355,0.045948,0.004465,0.002172,-0.018634,-0.019644,-0.007259,0.006397
0.025382,-0.024706,0.011424,0.028361,0.023778,0.008688,-0.006993,0.003408,-0.023627,0.007255,0.008234,-0.0094